In [1]:
import pandas as pd
import numpy as np
import utm
import matplotlib.pyplot as plt
from utils.preprocessing import preprocess
from utils.from_latlon import from_latlon
import math
from sklearn.linear_model import LinearRegression
import tkinter as tk
from tkinter import filedialog
from tkinter.simpledialog import askstring

C:\Users\s5236256\AppData\Local\Temp\ipykernel_10964\3939528926.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
routine = 'training'
dimensions = ['xOffset', 'yOffset']
sm = 150 # maximum signal difference between min and max, used for calibration

In [3]:
def preprocess_sim_data(sim_data, freq, tower_locs, routine):
    
    sim_dat_filt, predictors = preprocess(sim_data, freq, routine)
     
    # Calculate easting and northing from lat long
    sim_dat_filt['easting'], sim_dat_filt['northing'], sim_dat_filt['zone_num'], sim_dat_filt['zone_letter'] = from_latlon(sim_dat_filt['POINT_Y'].values, sim_dat_filt['POINT_X'].values)

    # Create a dictionary of the coordinates of the towers
    offset_dict = tower_locs.set_index('TowerID').to_dict()
    point_x = offset_dict['POINT_X']
    point_y = offset_dict['POINT_Y']

    # Standardise the coordinates so that the tower location == 0 on both the x and y axes.
    sim_dat_filt['xOffset'] = sim_dat_filt['easting'] - sim_dat_filt['TowerID'].map(point_x).fillna(0)
    sim_dat_filt['yOffset'] = sim_dat_filt['northing'] - sim_dat_filt['TowerID'].map(point_y).fillna(0)
    
    sim_dat_filt['easting_of_tower'] = sim_dat_filt['TowerID'].map(point_x).fillna(0)
    sim_dat_filt['northing_of_tower'] = sim_dat_filt['TowerID'].map(point_y).fillna(0)

    return sim_dat_filt, predictors

In [4]:
# User input of data paths and temporal resolution

# Initialize Tkinter
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

# Ask the user to select the train data file
train_data = filedialog.askopenfilename(
    title="Select training data",
    filetypes=[("Excel files", "*.xlsx")]
)

# Ask the user to select the test data file
test_data = filedialog.askopenfilename(
    title="Select testing data",
    filetypes=[("Excel files", "*.xlsx")]
)

# Ask the user to select the radio tower XY data file
radio_tower_xy_path = filedialog.askopenfilename(
    title="Select radio tower location data",
    filetypes=[("Excel files", "*.xlsx")]
)

# Ask the user to select the model save path
model_save_path = filedialog.askdirectory(
    title="Select model save path"
)

# Function to get minutes from user
def get_minutes():
    while True:
        minutes = askstring("Time (in minutes) to compile location data (t)", "Enter time period (t) in minutes (must be an integer):")
        if minutes and minutes.isdigit():
            return minutes
        messagebox.showerror("Error", "Invalid input. Please enter a number.")

# Prompt the user and get the validated input
minutes = get_minutes()

# Append the input number to 'min'
freq = minutes + 'min'

# Print freq to verify (optional)
print("Frequency:", freq)

Frequency: 3min


In [7]:
# Get training data
train_data = pd.read_excel(train_data)
train_data['DateAndTime'] = pd.to_datetime(train_data['DateAndTime'])

# Get testing data
test_data = pd.read_excel(test_data)
test_data['DateAndTime'] = pd.to_datetime(test_data['DateAndTime'])

# Get tower locations
tower_locs = pd.read_excel(radio_tower_xy_path)

# Get ML model location estimates (for comparison)
#ml_estimates_path = r'Example_data\Output\Predictions\UTM_predictions_combined_dtypes_input_20230701.xlsx'
#ml_estimates = pd.read_excel(ml_estimates_path)

In [8]:
# Create test dataset as per train_model_h2o method
test_data_preproc, predictors_test = preprocess_sim_data(test_data, freq, tower_locs, routine)
train_data_preproc, predictors_train = preprocess_sim_data(train_data, freq, tower_locs, routine)

c:\Users\s5236256\Documents\GitHub\ml4rt\utils\preprocessing.py:10: FutureWarning: The provided callable <function std at 0x0000025F5B478670> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .agg(['mean', 'count', np.std])
c:\Users\s5236256\Documents\GitHub\ml4rt\utils\preprocessing.py:10: FutureWarning: The provided callable <function std at 0x0000025F5B478670> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .agg(['mean', 'count', np.std])


In [9]:
def calculate_delta_g(s1, s2, sm):
    delta_g = (s1-s2)/sm
    return delta_g

def calculate_offset_angle(delta_g):
    bearing = math.acos(delta_g) * 90/math.pi
    return bearing

In [10]:
# Make distance values absolute
train_data_preproc['xOffset_abs'] = train_data_preproc['xOffset'].abs()
train_data_preproc['yOffset_abs'] = train_data_preproc['yOffset'].abs()

# Perform linear regression for each antenna
train_data_filtered = train_data_preproc[train_data_preproc['ant1_mean'] != 0]
x = train_data_filtered[['ant1_mean']]
y = train_data_filtered[['yOffset_abs']]
model_ant1 = LinearRegression().fit(x, y)

train_data_filtered = train_data_preproc[train_data_preproc['ant2_mean'] != 0]
x = train_data_filtered[['ant2_mean']]
y = train_data_filtered[['xOffset_abs']]
model_ant2 = LinearRegression().fit(x, y)

train_data_filtered = train_data_preproc[train_data_preproc['ant3_mean'] != 0]
x = train_data_filtered[['ant3_mean']]
y = train_data_filtered[['yOffset_abs']]
model_ant3 = LinearRegression().fit(x, y)

train_data_filtered = train_data_preproc[train_data_preproc['ant4_mean'] != 0]
x = train_data_filtered[['ant4_mean']]
y = train_data_filtered[['xOffset_abs']]
model_ant4 = LinearRegression().fit(x, y)

# Also create an average model
# Extract coefficients from individual models
coefficients_ant1 = np.array([model_ant1.coef_[0][0], model_ant1.intercept_[0]])
coefficients_ant2 = np.array([model_ant2.coef_[0][0], model_ant2.intercept_[0]])
coefficients_ant3 = np.array([model_ant3.coef_[0][0], model_ant3.intercept_[0]])
coefficients_ant4 = np.array([model_ant4.coef_[0][0], model_ant4.intercept_[0]])

# Calculate average coefficients
average_coefficients = np.mean([coefficients_ant1, coefficients_ant2, coefficients_ant3, coefficients_ant4], axis=0)

# Create a new Linear Regression model with average coefficients
model_ant_average = LinearRegression()
model_ant_average.coef_ = np.array([average_coefficients[0]])
model_ant_average.intercept_ = np.array([average_coefficients[1]])

In [11]:
## Calculate total paired antenna values to find maximum sum, only if both values do not equal zero.

# Add 'ant_1_2_sum' column
test_data_preproc['ant_1_2_sum'] = test_data_preproc.apply(lambda row: row['ant1_mean'] + row['ant2_mean'] if row['ant1_mean'] != 0 and row['ant2_mean'] != 0 else 'na', axis=1)

# Add 'ant_2_3_sum' column
test_data_preproc['ant_2_3_sum'] = test_data_preproc.apply(lambda row: row['ant2_mean'] + row['ant3_mean'] if row['ant2_mean'] != 0 and row['ant3_mean'] != 0 else 'na', axis=1)

# Add 'ant_3_4_sum' column
test_data_preproc['ant_3_4_sum'] = test_data_preproc.apply(lambda row: row['ant3_mean'] + row['ant4_mean'] if row['ant3_mean'] != 0 and row['ant4_mean'] != 0 else 'na', axis=1)

# Add 'ant_4_1_sum' column
test_data_preproc['ant_4_1_sum'] = test_data_preproc.apply(lambda row: row['ant4_mean'] + row['ant1_mean'] if row['ant4_mean'] != 0 and row['ant1_mean'] != 0 else 'na', axis=1)



In [12]:
#### Calculate the bearings from the tower

# List of antenna names
antenna_names = ['ant_1_2_sum', 'ant_2_3_sum', 'ant_3_4_sum', 'ant_4_1_sum']

# Create a new column 'bearing' in the DataFrame
test_data_preproc['bearing'] = None

# Iterate through the DataFrame
for index, row in test_data_preproc.iterrows():
    # Extract values from the new columns and handle 'na' values
    values = [float(row['ant_1_2_sum']) if row['ant_1_2_sum'] != 'na' else float('-inf'),
              float(row['ant_2_3_sum']) if row['ant_2_3_sum'] != 'na' else float('-inf'),
              float(row['ant_3_4_sum']) if row['ant_3_4_sum'] != 'na' else float('-inf'),
              float(row['ant_4_1_sum']) if row['ant_4_1_sum'] != 'na' else float('-inf')]
    
    # Check if all values are 'na'; if yes, skip the row
    if all(val == float('-inf') for val in values):
        continue
    
    # Find the column name with the highest value
    max_column = max(zip(values, antenna_names))[1]
        
    if max_column == "ant_1_2_sum":
        if row.ant1_mean >= row.ant2_mean:
            delta_g = calculate_delta_g(row.ant1_mean, row.ant2_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)
            bearing = offset_angle
            
            # print(f'delta_g = {delta_g}')
            # print(f'offset_angle from ant 1 = {offset_angle}')
            # print(f'bearing ant 1 = {bearing}')

        else:
            delta_g = calculate_delta_g(row.ant2_mean, row.ant1_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)          
            bearing = 90 - offset_angle

    elif max_column == "ant_2_3_sum":
        if row.ant2_mean >= row.ant3_mean:
            delta_g = calculate_delta_g(row.ant2_mean, row.ant3_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)
            bearing = 90 + offset_angle

        else:
            delta_g = calculate_delta_g(row.ant3_mean, row.ant2_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)          
            bearing = 180 - offset_angle

    elif max_column == "ant_3_4_sum":
        if row.ant3_mean >= row.ant4_mean:
            delta_g = calculate_delta_g(row.ant3_mean, row.ant4_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)
            bearing = 180 + offset_angle

        else:
            delta_g = calculate_delta_g(row.ant4_mean, row.ant3_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)          
            bearing = 270 - offset_angle

    elif max_column == "ant_4_1_sum":
        if row.ant4_mean >= row.ant1_mean:
            delta_g = calculate_delta_g(row.ant4_mean, row.ant1_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)
            bearing = 270 + offset_angle

        else:
            delta_g = calculate_delta_g(row.ant1_mean, row.ant4_mean, sm)
            offset_angle = calculate_offset_angle(delta_g)          
            bearing = 360 - offset_angle
    
    
    # Correct bearings for magnetic declination and handle those that go over 360 degrees
    bearing = bearing + 7.6
    if bearing >= 360:
        bearing = bearing - 360
    else:
        pass

    # Assign the calculated bearing to the 'bearing' column
    test_data_preproc.at[index, 'bearing'] = bearing

In [13]:
# Convert non-numeric values to NaN
numeric_columns = ['ant_1_2_sum', 'ant_2_3_sum', 'ant_3_4_sum', 'ant_4_1_sum']
test_data_preproc[numeric_columns] = test_data_preproc[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Calculate the maximum value across columns while handling NaN values
test_data_preproc['max_value'] = np.nanmax(test_data_preproc[numeric_columns].values, axis=1) / 2


C:\Users\s5236256\AppData\Local\Temp\ipykernel_10964\4233910366.py:6: RuntimeWarning: All-NaN slice encountered
  test_data_preproc['max_value'] = np.nanmax(test_data_preproc[numeric_columns].values, axis=1) / 2


In [14]:
test_data_preproc

,DateTime,TowerID,TagID,Data_type,POINT_X,POINT_Y,Point_ID,Interval_seconds,ant1_count,ant2_count,...,xOffset,yOffset,easting_of_tower,northing_of_tower,ant_1_2_sum,ant_2_3_sum,ant_3_4_sum,ant_4_1_sum,bearing,max_value
0,2021-02-02 07:57:00,RT04,60,BTFS,146.255722,-21.919832,300,13,4.0,1.0,...,184.203413,186.159797,422946.117861,7.575674e+06,279.000000,262.500000,238.833333,255.333333,50.880094,139.500000
1,2021-02-02 08:00:00,RT04,60,BTFS,146.255722,-21.919832,300,13,1.0,2.0,...,184.203413,186.159797,422946.117861,7.575674e+06,281.000000,257.000000,262.000000,286.000000,326.816902,143.000000
2,2021-09-15 14:54:00,RT12,101,BTFS,146.295848,-22.011675,7,13,3.0,0.0,...,76.787542,65.239343,427244.674540,7.565648e+06,NaN,NaN,132.166667,152.166667,331.362176,76.083333
3,2021-09-15 14:54:00,RT13,101,BTFS,146.295848,-22.011675,7,13,3.0,0.0,...,102.946773,450.817060,427218.515309,7.565263e+06,NaN,NaN,NaN,NaN,None,NaN
4,2021-09-15 14:54:00,RT15,101,BTFS,146.295848,-22.011675,7,13,0.0,2.0,...,-342.056603,86.762302,427663.518685,7.565627e+06,NaN,NaN,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2023-05-23 13:00:00,RT18,192,BTFS,146.235674,-21.936865,546,3,10.0,9.0,...,-60.686126,-22.545451,421129.737817,7.573987e+06,215.488889,229.688889,250.300000,236.100000,238.115303,125.150000
396,2023-05-23 13:03:00,RT18,192,BTFS,146.235674,-21.936865,546,3,5.0,5.0,...,-60.686126,-22.545451,421129.737817,7.573987e+06,204.800000,227.400000,226.050000,203.450000,143.402246,113.700000
397,2023-05-23 13:36:00,RT18,211,BTFS,146.236103,-21.939429,548,3,0.0,0.0,...,-14.968154,-306.139731,421129.737817,7.573987e+06,NaN,NaN,NaN,NaN,None,NaN
398,2023-05-23 13:39:00,RT18,211,BTFS,146.236103,-21.939429,548,3,13.0,4.0,...,-14.968154,-306.139731,421129.737817,7.573987e+06,148.673077,161.750000,NaN,NaN,149.986793,80.875000


In [15]:
# Calculate locations without using biangulation

# List of antenna names
antennas = ['ant1_mean', 'ant2_mean', 'ant3_mean', 'ant4_mean']

for index, row in test_data_preproc.iterrows():
    # First calculate appropriate bearing and infer distance using linear model
    if row['bearing'] != None:
        distance = model_ant_average.predict(np.array([[row['max_value']]]))[0]
        bearing = row['bearing']
        test_data_preproc.at[index, 'Mech_method'] = "Multiple antenna, single tower"

    elif row['bearing'] == None:
        # Extract values from the new columns and handle 'na' values
        values = [float(row['ant1_mean']) if row['ant1_mean'] != 'na' else float('-inf'),
                float(row['ant2_mean']) if row['ant2_mean'] != 'na' else float('-inf'),
                float(row['ant3_mean']) if row['ant3_mean'] != 'na' else float('-inf'),
                float(row['ant4_mean']) if row['ant4_mean'] != 'na' else float('-inf')]

        test_data_preproc.at[index, 'Mech_method'] = "Single antenna, single tower"

        # Find the column name with the highest value
        max_single_ant = max(zip(values, antennas))[1]

        if max_single_ant == "ant1_mean":
            bearing = 0
            distance = model_ant1.predict(np.array([[row['ant1_mean']]]))[0]

        elif max_single_ant == "ant2_mean":
            bearing = 90
            distance = model_ant2.predict(np.array([[row['ant2_mean']]]))[0]

        elif max_single_ant == "ant3_mean":
            bearing = 180
            distance = model_ant3.predict(np.array([[row['ant3_mean']]]))[0]

        elif max_single_ant == "ant4_mean":
            bearing = 270
            distance = model_ant4.predict(np.array([[row['ant4_mean']]]))[0]

    else:
        pass
    
    # Correct bearings for magnetic declination and handle those that go over 360 degrees
    bearing = bearing + 7.6
    if bearing >= 360:
        bearing = bearing - 360
    else:
        pass
    
    # Estimating new location using trigonometry
    bearing_rad = math.radians(bearing)

    # Calculate new coordinates
    new_easting = row['easting_of_tower'] + distance * math.sin(bearing_rad)
    new_northing = row['northing_of_tower'] + distance * math.cos(bearing_rad)

    # Assigning the new coordinates to the 'new_easting' and 'new_northing' columns in test_data_preproc
    test_data_preproc.at[index, 'mech_bearing_easting'] = new_easting
    test_data_preproc.at[index, 'mech_bearing_northing'] = new_northing



c:\Users\s5236256\Documents\GitHub\ml4rt\.venv\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\s5236256\Documents\GitHub\ml4rt\.venv\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\s5236256\Documents\GitHub\ml4rt\.venv\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\s5236256\Documents\GitHub\ml4rt\.venv\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\s5236256\Documents\GitHub\ml4rt\.venv\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature nam

In [16]:
test_data_preproc

,DateTime,TowerID,TagID,Data_type,POINT_X,POINT_Y,Point_ID,Interval_seconds,ant1_count,ant2_count,...,northing_of_tower,ant_1_2_sum,ant_2_3_sum,ant_3_4_sum,ant_4_1_sum,bearing,max_value,Mech_method,mech_bearing_easting,mech_bearing_northing
0,2021-02-02 07:57:00,RT04,60,BTFS,146.255722,-21.919832,300,13,4.0,1.0,...,7.575674e+06,279.000000,262.500000,238.833333,255.333333,50.880094,139.500000,"Multiple antenna, single tower",422908.157044,7.575651e+06
1,2021-02-02 08:00:00,RT04,60,BTFS,146.255722,-21.919832,300,13,1.0,2.0,...,7.575674e+06,281.000000,257.000000,262.000000,286.000000,326.816902,143.000000,"Multiple antenna, single tower",422975.529678,7.575612e+06
2,2021-09-15 14:54:00,RT12,101,BTFS,146.295848,-22.011675,7,13,3.0,0.0,...,7.565648e+06,NaN,NaN,132.166667,152.166667,331.362176,76.083333,"Multiple antenna, single tower",427107.283171,7.566005e+06
3,2021-09-15 14:54:00,RT13,101,BTFS,146.295848,-22.011675,7,13,3.0,0.0,...,7.565263e+06,NaN,NaN,NaN,NaN,None,NaN,"Single antenna, single tower",427267.988394,7.565633e+06
4,2021-09-15 14:54:00,RT15,101,BTFS,146.295848,-22.011675,7,13,0.0,2.0,...,7.565627e+06,NaN,NaN,NaN,NaN,None,NaN,"Single antenna, single tower",427393.514233,7.565663e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2023-05-23 13:00:00,RT18,192,BTFS,146.235674,-21.936865,546,3,10.0,9.0,...,7.573987e+06,215.488889,229.688889,250.300000,236.100000,238.115303,125.150000,"Multiple antenna, single tower",421082.203791,7.573966e+06
396,2023-05-23 13:03:00,RT18,192,BTFS,146.235674,-21.936865,546,3,5.0,5.0,...,7.573987e+06,204.800000,227.400000,226.050000,203.450000,143.402246,113.700000,"Multiple antenna, single tower",421192.414388,7.573874e+06
397,2023-05-23 13:36:00,RT18,211,BTFS,146.236103,-21.939429,548,3,0.0,0.0,...,7.573987e+06,NaN,NaN,NaN,NaN,None,NaN,"Single antenna, single tower",421098.967685,7.573756e+06
398,2023-05-23 13:39:00,RT18,211,BTFS,146.236103,-21.939429,548,3,13.0,4.0,...,7.573987e+06,148.673077,161.750000,NaN,NaN,149.986793,80.875000,"Multiple antenna, single tower",421263.354799,7.573663e+06


In [17]:
# Replace 'None' and 'na' with NaN in the 'bearing' column
test_data_preproc['bearing'].replace(['None', 'na'], float('nan'), inplace=True)

# Drop rows with any missing values in 'bearing' column
test_data_preproc_nona = test_data_preproc.dropna(subset=['bearing'], how='any')

C:\Users\s5236256\AppData\Local\Temp\ipykernel_10964\1673892558.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_data_preproc['bearing'].replace(['None', 'na'], float('nan'), inplace=True)
C:\Users\s5236256\AppData\Local\Temp\ipykernel_10964\1673892558.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcastin

In [18]:
def triang(x1, y1, alpha1, x2, y2, alpha2):
    # For Triangulation GK Coordinates are necessary!
    # First calculate tan keeping in mind that 0° in geo-coordinates are 90° in an x-y plane
    ta1 = (alpha1 % 360) / 180 * np.pi
    ta2 = (alpha2 % 360) / 180 * np.pi

    if ((alpha1 - alpha2) % 180 == 0):
        # print("No triangulation possible: all three points are on one line")
        return (np.nan, np.nan)

    # Finding Intersection Using solver
    b = np.array([x2 - x1, y2 - y1])
    a1 = np.array([np.sin(ta1), np.cos(ta1)])
    a2 = np.array([-np.sin(ta2), -np.cos(ta2)])
    a = np.vstack((a1, a2))
    
    try:
        l = np.linalg.solve(a, b)
    except np.linalg.LinAlgError:
        return (np.nan, np.nan)

    px = x1 + l[0] * np.sin(ta1)
    py = y1 + l[0] * np.cos(ta1)

    if l[1] > 0 and l[0] > 0:
        return px, py
    else:
        return np.nan, np.nan



In [19]:
def apply_triangulation(df):
    result_list = []

    # Group by 'TagID' and 'DateTime'
    grouped = df.groupby(['TagID', 'DateTime'])

    for name, group in grouped:
        # For each unique pair of 'TowerID'
        tower_ids = group['TowerID'].unique()

        # Skip if there's only one tower or missing data
        if len(tower_ids) < 2 or group['bearing'].isnull().any():
            continue

        # Generate all unique pairs of TowerID
        tower_combinations = np.array(np.meshgrid(tower_ids, tower_ids)).T.reshape(-1, 2)

        for towers in tower_combinations:
            # Extract data for each TowerID pair
            data1 = group[group['TowerID'] == towers[0]]
            data2 = group[group['TowerID'] == towers[1]]

            # Extract x, y, and alpha values
            x1, y1, alpha1 = data1['easting_of_tower'].values[0], data1['northing_of_tower'].values[0], data1['bearing'].values[0]
            x2, y2, alpha2 = data2['easting_of_tower'].values[0], data2['northing_of_tower'].values[0], data2['bearing'].values[0]

            # Apply triangulation
            easting_estimated, northing_estimated = triang(x1, y1, alpha1, x2, y2, alpha2)
            
            # Append the result along with TagID, DateTime, and TowerID information
            result_list.append({'TagID': name[0], 'DateTime': name[1], 'TowerID1': towers[0], 'TowerID2': towers[1],
                                'mech_tri_easting': easting_estimated, 'mech_tri_northing': northing_estimated})

    # Create a DataFrame from the results
    result_df = pd.DataFrame(result_list)

    return result_df

# Apply triangulation to your data
result_dataframe = apply_triangulation(test_data_preproc_nona)

# Print or use result_dataframe as needed
print(result_dataframe)


    TagID            DateTime TowerID1 TowerID2  mech_tri_easting  \
0     100 2021-09-17 10:33:00     RT12     RT12               NaN   
1     100 2021-09-17 10:33:00     RT12     RT15               NaN   
2     100 2021-09-17 10:33:00     RT15     RT12               NaN   
3     100 2021-09-17 10:33:00     RT15     RT15               NaN   
4     190 2023-02-26 09:24:00     RT13     RT13               NaN   
..    ...                 ...      ...      ...               ...   
65    209 2023-05-10 11:12:00     RT14     RT14               NaN   
66    214 2023-05-20 10:09:00     RT01     RT01               NaN   
67    214 2023-05-20 10:09:00     RT01     RT02               NaN   
68    214 2023-05-20 10:09:00     RT02     RT01               NaN   
69    214 2023-05-20 10:09:00     RT02     RT02               NaN   

    mech_tri_northing  
0                 NaN  
1                 NaN  
2                 NaN  
3                 NaN  
4                 NaN  
..                ...  
65 

In [20]:
# Merge the triangulation data back in with the bearing estimates
mech_bearing_and_tri_estimates = pd.merge(test_data_preproc, result_dataframe, on=['TagID', 'DateTime'], how='left')


In [21]:
mech_bearing_and_tri_estimates

,DateTime,TowerID,TagID,Data_type,POINT_X,POINT_Y,Point_ID,Interval_seconds,ant1_count,ant2_count,...,ant_4_1_sum,bearing,max_value,Mech_method,mech_bearing_easting,mech_bearing_northing,TowerID1,TowerID2,mech_tri_easting,mech_tri_northing
0,2021-02-02 07:57:00,RT04,60,BTFS,146.255722,-21.919832,300,13,4.0,1.0,...,255.333333,50.880094,139.500000,"Multiple antenna, single tower",422908.157044,7.575651e+06,NaN,NaN,NaN,NaN
1,2021-02-02 08:00:00,RT04,60,BTFS,146.255722,-21.919832,300,13,1.0,2.0,...,286.000000,326.816902,143.000000,"Multiple antenna, single tower",422975.529678,7.575612e+06,NaN,NaN,NaN,NaN
2,2021-09-15 14:54:00,RT12,101,BTFS,146.295848,-22.011675,7,13,3.0,0.0,...,152.166667,331.362176,76.083333,"Multiple antenna, single tower",427107.283171,7.566005e+06,NaN,NaN,NaN,NaN
3,2021-09-15 14:54:00,RT13,101,BTFS,146.295848,-22.011675,7,13,3.0,0.0,...,NaN,NaN,NaN,"Single antenna, single tower",427267.988394,7.565633e+06,NaN,NaN,NaN,NaN
4,2021-09-15 14:54:00,RT15,101,BTFS,146.295848,-22.011675,7,13,0.0,2.0,...,NaN,NaN,NaN,"Single antenna, single tower",427393.514233,7.565663e+06,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,2023-05-23 13:00:00,RT18,192,BTFS,146.235674,-21.936865,546,3,10.0,9.0,...,236.100000,238.115303,125.150000,"Multiple antenna, single tower",421082.203791,7.573966e+06,NaN,NaN,NaN,NaN
634,2023-05-23 13:03:00,RT18,192,BTFS,146.235674,-21.936865,546,3,5.0,5.0,...,203.450000,143.402246,113.700000,"Multiple antenna, single tower",421192.414388,7.573874e+06,NaN,NaN,NaN,NaN
635,2023-05-23 13:36:00,RT18,211,BTFS,146.236103,-21.939429,548,3,0.0,0.0,...,NaN,NaN,NaN,"Single antenna, single tower",421098.967685,7.573756e+06,NaN,NaN,NaN,NaN
636,2023-05-23 13:39:00,RT18,211,BTFS,146.236103,-21.939429,548,3,13.0,4.0,...,NaN,149.986793,80.875000,"Multiple antenna, single tower",421263.354799,7.573663e+06,NaN,NaN,NaN,NaN


In [22]:
# Excel data export to work out why bearings could not be calculated

mech_bearing_and_tri_estimates.to_excel('SBTF_data/ml4rt_output/trained_models/mech_model/mech_raw_bearings_working_20240127.xlsx', index=False)

In [29]:
mech_pos_estimates = mech_bearing_and_tri_estimates.groupby(['TagID', 'DateTime']).agg({
    'mech_bearing_easting': 'mean', 
    'mech_bearing_northing': 'mean', 
    'mech_tri_easting': 'mean', 
    'mech_tri_northing': 'mean',
    'easting': 'first',
    'northing': 'first',
    'Data_type': 'first'
}).reset_index()

In [28]:
mech_bearing_and_tri_estimates.columns

Index(['DateTime', 'TowerID', 'TagID', 'Data_type', 'POINT_X', 'POINT_Y',
       'Point_ID', 'Interval_seconds', 'ant1_count', 'ant2_count',
       'ant3_count', 'ant4_count', 'ant1_mean', 'ant2_mean', 'ant3_mean',
       'ant4_mean', 'ant1_std', 'ant2_std', 'ant3_std', 'ant4_std', 'mean_std',
       'total_count', 'easting', 'northing', 'zone_num', 'zone_letter',
       'xOffset', 'yOffset', 'easting_of_tower', 'northing_of_tower',
       'ant_1_2_sum', 'ant_2_3_sum', 'ant_3_4_sum', 'ant_4_1_sum', 'bearing',
       'max_value', 'Mech_method', 'mech_bearing_easting',
       'mech_bearing_northing', 'TowerID1', 'TowerID2', 'mech_tri_easting',
       'mech_tri_northing'],
      dtype='object')

In [30]:
mech_pos_estimates

,TagID,DateTime,mech_bearing_easting,mech_bearing_northing,mech_tri_easting,mech_tri_northing,easting,northing,Data_type
0,60,2021-02-02 07:57:00,422908.157044,7.575651e+06,NaN,NaN,423130.321274,7.575860e+06,BTFS
1,60,2021-02-02 08:00:00,422975.529678,7.575612e+06,NaN,NaN,423130.321274,7.575860e+06,BTFS
2,92,2021-09-18 18:09:00,436788.746501,7.551655e+06,NaN,NaN,436997.613395,7.551066e+06,BTFS
3,92,2021-09-18 18:12:00,436764.569158,7.551783e+06,NaN,NaN,436997.613395,7.551066e+06,BTFS
4,92,2021-09-18 18:15:00,436757.522887,7.551823e+06,NaN,NaN,436997.613395,7.551066e+06,BTFS
...,...,...,...,...,...,...,...,...,...
171,222,2023-02-27 18:09:00,421364.669203,7.574081e+06,NaN,NaN,421339.668957,7.573978e+06,BTFS
172,222,2023-02-27 18:12:00,421364.451469,7.574082e+06,NaN,NaN,421339.668957,7.573978e+06,BTFS
173,222,2023-02-27 18:15:00,421375.637531,7.574082e+06,NaN,NaN,421339.668957,7.573978e+06,BTFS
174,222,2023-02-27 18:18:00,421357.423914,7.574078e+06,NaN,NaN,421339.668957,7.573978e+06,BTFS


In [31]:
# Merge result_dataframe with ml_estimates
mech_pos_estimates['mech_tri_easting_error'] = mech_pos_estimates['mech_tri_easting'] - mech_pos_estimates['easting']
mech_pos_estimates['mech_tri_northing_error'] = mech_pos_estimates['mech_tri_northing'] - mech_pos_estimates['northing']

mech_pos_estimates['mech_bearing_easting_error'] = mech_pos_estimates['mech_bearing_easting'] - mech_pos_estimates['easting']
mech_pos_estimates['mech_bearing_northing_error'] = mech_pos_estimates['mech_bearing_northing'] - mech_pos_estimates['northing']

# Calculate the Eucledian distance between the predicted and actual locations
mech_pos_estimates['error_mech_tri_m'] = np.sqrt((mech_pos_estimates['mech_tri_easting_error']) ** 2
                        + (mech_pos_estimates['mech_tri_northing_error']) ** 2)

mech_pos_estimates['error_mech_bearing_m'] = np.sqrt((mech_pos_estimates['mech_bearing_easting_error']) ** 2
                        + (mech_pos_estimates['mech_bearing_northing_error']) ** 2)

mech_pos_estimates

,TagID,DateTime,mech_bearing_easting,mech_bearing_northing,mech_tri_easting,mech_tri_northing,easting,northing,Data_type,mech_tri_easting_error,mech_tri_northing_error,mech_bearing_easting_error,mech_bearing_northing_error,error_mech_tri_m,error_mech_bearing_m
0,60,2021-02-02 07:57:00,422908.157044,7.575651e+06,NaN,NaN,423130.321274,7.575860e+06,BTFS,NaN,NaN,-222.164231,-209.440361,NaN,305.323124
1,60,2021-02-02 08:00:00,422975.529678,7.575612e+06,NaN,NaN,423130.321274,7.575860e+06,BTFS,NaN,NaN,-154.791596,-247.593496,NaN,291.998249
2,92,2021-09-18 18:09:00,436788.746501,7.551655e+06,NaN,NaN,436997.613395,7.551066e+06,BTFS,NaN,NaN,-208.866894,589.146947,NaN,625.075599
3,92,2021-09-18 18:12:00,436764.569158,7.551783e+06,NaN,NaN,436997.613395,7.551066e+06,BTFS,NaN,NaN,-233.044237,717.440595,NaN,754.341186
4,92,2021-09-18 18:15:00,436757.522887,7.551823e+06,NaN,NaN,436997.613395,7.551066e+06,BTFS,NaN,NaN,-240.090508,757.551693,NaN,794.687372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,222,2023-02-27 18:09:00,421364.669203,7.574081e+06,NaN,NaN,421339.668957,7.573978e+06,BTFS,NaN,NaN,25.000245,102.778637,NaN,105.775520
172,222,2023-02-27 18:12:00,421364.451469,7.574082e+06,NaN,NaN,421339.668957,7.573978e+06,BTFS,NaN,NaN,24.782512,103.321124,NaN,106.251718
173,222,2023-02-27 18:15:00,421375.637531,7.574082e+06,NaN,NaN,421339.668957,7.573978e+06,BTFS,NaN,NaN,35.968573,104.037469,NaN,110.079668
174,222,2023-02-27 18:18:00,421357.423914,7.574078e+06,NaN,NaN,421339.668957,7.573978e+06,BTFS,NaN,NaN,17.754956,99.727738,NaN,101.295904


In [32]:
mech_pos_estimates.columns


Index(['TagID', 'DateTime', 'mech_bearing_easting', 'mech_bearing_northing',
       'mech_tri_easting', 'mech_tri_northing', 'easting', 'northing',
       'Data_type', 'mech_tri_easting_error', 'mech_tri_northing_error',
       'mech_bearing_easting_error', 'mech_bearing_northing_error',
       'error_mech_tri_m', 'error_mech_bearing_m'],
      dtype='object')

In [82]:
# Excel data export
merged_dataframe.to_excel('Example_data\Output\Predictions\mech_ml_estimates_mag_decl_incl_20231112.xlsx', index=False)